Using a 1D CNN Autoencoder to cluster column maxes for each track, to cluster together protons + deuterons and noise. 

Data created in **padded_data.ipynb** which uses data created in **clusters_data.ipynb**.


In [2]:
import numpy as np

In [3]:
PATH_ALL = "/Users/user/data/research/proton-deuteron/csv/padded_max_adc_columns.npy"
PROTONS = '/Users/user/data/research/proton-deuteron/csv/padded_protons.npy'
DEUTERONS = '/Users/user/data/research/proton-deuteron/csv/padded_deuterons.npy'

In [11]:
# threshold: 15 ADC, for making clusters of connected pixels.
# then, max ADC of each wire in cluster is recorded only in a 1D matrix.
# then this array is padded with zeros on either side, with original wire coordinates preserved.
# cut: wire lengths > 10 only, mixed collection, induction, proton, deuteron candidates.
# full data, pre-training/testing split. 
# 1D data, (1x240), 70922 total data points (16097 proton candidates, 54825 deuterons candidates), collection, induction mixed.   

padded_protons = np.load(PROTONS) 
padded_deuterons = np.load(DEUTERONS)